In [ ]:
# /content/drive/MyDrive/NLP and Gen-AI 13 Jan 24  /Daily Class Notes/24_03_27_/output_dir/checkpoint-1000

##Dataset loading

In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv("/content/drive/MyDrive/NLP and Gen-AI 13 Jan 24  /Daily Class Notes/24_02_27_Project/complaints.csv")

In [8]:
df = df[["Product", "Consumer complaint narrative"]]

In [9]:
df = df.dropna()

In [10]:
class_dict =  {

'Vehicle loan or lease' : 'loan',
'Credit reporting, credit repair services, or other personal consumer reports' : 'credit_report',
'Credit card or prepaid card' : 'card' ,
'Money transfer, virtual currency, or money service' : 'money_transfer',
'Mortgage' : 'Mortgage',
'Payday loan, title loan, or personal loan' : 'loan' ,
'Debt collection' : 'Debt collection',
'Checking or savings account' : 'account',
'Student loan' : 'loan',
'Consumer Loan' : 'loan',
'Money transfers' : 'money_transfer',
'Credit card' : 'card',
'Bank account or service' : 'account',
'Credit reporting' : 'credit_report',
'Prepaid card' : 'card',
'Payday loan' : 'loan',
'Other financial service' : 'credit_report',
'Virtual currency' : 'money_transfer'
}

In [11]:
df.replace({'Product': class_dict}, inplace=True)

In [12]:
sampled_df = pd.DataFrame()

for col in df['Product'].unique():
  sample = df[df['Product'] == col].sample(1000)

  sampled_df = sampled_df.append(sample, ignore_index=True)
  # print(col)

<ipython-input-12-57337da2bbd5>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sampled_df = sampled_df.append(sample, ignore_index=True)


In [13]:
import re

In [14]:
def text_cleaning(text):
  text = text.lower()   #lower case

  text = re.sub(r'[^\w\s]', '', text)  #removing punct

  text = re.sub(r'[0-9]', '', text)   #removing digits

  text = re.sub(r'[x]{2,}', '', text) # removing more than 2 occurance of X

  return text

In [15]:
sampled_df['Consumer complaint narrative'] = sampled_df['Consumer complaint narrative'].apply(text_cleaning)

In [16]:
classes = {'loan' : 0,
           'credit_report' :1,
           'card': 2,
           'money_transfer' : 3,
           'Mortgage' : 4,
           'Debt collection' : 5,
           'account' : 6
          }

sampled_df.replace({'Product' : classes}, inplace=True)

In [17]:
sampled_df.columns = ['label', 'text']

In [18]:
sampled_df.to_csv("complaint_dataset.csv", index=False)

In [19]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [20]:
from datasets import load_dataset

In [21]:
data = load_dataset('csv', data_files='complaint_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
train_test = data['train'].train_test_split(test_size=0.2, seed=42)

In [24]:
train_test['test']

Dataset({
    features: ['label', 'text'],
    num_rows: 1400
})

##load fine tuned model

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

In [3]:
model_checkpoint = "/content/drive/MyDrive/NLP and Gen-AI 13 Jan 24  /Daily Class Notes/24_03_27_/output_dir/checkpoint-1000"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

In [5]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [26]:
train_test['test']['text'][:1]

['good day my name is   and i currently have a mortgage loan with citi mortgage in the city of  my property taxes were lower by  by the city of  documentation was sent over to citi mortgage and only to find out my mortgage will only be lower by  for the year i am current with my mortgage so i do not understand their calculation i called the company and they are lying about escrow differences i was told the same thing by citi staff last year when they raised my mortgage from  to  i really need your help to correct this matter because i know i will lose my home with citi mortgage scheme thank you']

In [29]:
train_test['test']['label'][:1]

[4]

In [27]:
predz = classifier(train_test['test']['text'][:100], padding=True, truncation=True)

In [ ]:
predz

In [36]:
y_pred = [int(label['label'][-1]) for label in predz]

In [ ]:
y_pred

In [ ]:
for label in predz:
  print(int(label['label'][-1]))

In [ ]:
predz

In [35]:
y_actual = train_test['test']['label'][:100]

In [37]:
from sklearn.metrics import accuracy_score, f1_score

In [38]:
accuracy_score(y_actual, y_pred)

0.8

In [40]:
f1_score(y_actual, y_pred, average='macro')

0.802880645238878

In [ ]:
## Business impact

In [42]:
train_test['test']['label'][:1]

[4]

In [44]:
complaint = train_test['test']['text'][:1]

prediction = classifier(complaint, padding=True, truncation=True)

prediction

[{'label': 'LABEL_4', 'score': 0.9914877414703369}]

In [45]:
prediction[0]['label']

'LABEL_4'

In [51]:
def get_receiver_email(prediction):
  if prediction[0]['label'] == 'LABEL_0':
    receiver_email = 'jadhavanil877@gmail.com'

  elif prediction[0]['label'] == 'LABEL_1':
    receiver_email = 'info.sangram@yahoo.com'

  elif prediction[0]['label'] == 'LABEL_2':
    receiver_email = 'ameyshejwal8903@gmail.com'

  elif prediction[0]['label'] == 'LABEL_3':
    receiver_email = 'ameyshejwal8903@gmail.com'

  elif prediction[0]['label'] == 'LABEL_4':
    receiver_email = 'snehaltapkire23@gmail.com'

  elif prediction[0]['label'] == 'LABEL_5':
    receiver_email = 'suryakantdawkhar53@gmail.com'

  elif prediction[0]['label'] == 'LABEL_6':
    receiver_email = 'shivpalw.nlpgenai@gmail.com'

  return receiver_email

In [47]:
receiver_email

'snehaltapkire23@gmail.com'

In [ ]:
# {'label': 'LABEL_0', 'score': 0.9703946709632874},


# classes = {'loan' : 0,         --> jadhavanil877@gmail.com
#            'credit_report' :1,  ---> info.sangram@yahoo.com
#            'card': 2,           ---> ameyshejwal8903@gmail.com
#            'money_transfer' : 3, --> atulmisal08@gmail.com
#            'Mortgage' : 4,         --> snehaltapkire23@gmail.com
#            'Debt collection' : 5,  --> suryakantdawkhar53@gmail.com
#            'account' : 6         --> shivpalw.nlpgenai@gmail.com
#           }

##Sending Mail

In [48]:
from google.colab import userdata

sender_password = userdata.get('EMAIL_PASSWORD')

In [64]:
import smtplib, ssl


def send_mail(receiver_email, message):
  port = 465
  smtp_server = 'smtp.gmail.com'

  sender_email = 'ugaledatta41@gmail.com'

  # receiver_email =

  # sender_password = ''
  # message = 'this mail send from python'

  context = ssl.create_default_context()

  server = smtplib.SMTP_SSL(smtp_server, port)

  server.login(sender_email, sender_password)

  server.sendmail(sender_email, receiver_email, message)

  print('mail sent')

In [66]:
def forward_complaint(complaint):
  prediction = classifier(complaint, padding=True, truncation=True)
  print(prediction)

  receiver_email = get_receiver_email(prediction)
  print(receiver_email)
  # server.sendmail(sender_email, receiver_email, complaint[0])
  send_mail(receiver_email, complaint[0])

  return {"message" : "complaint forwarded"}



complaint = train_test['test']['text'][100:101]

forward_complaint(complaint)

[{'label': 'LABEL_2', 'score': 0.9093519449234009}]
ameyshejwal8903@gmail.com
mail sent


{'message': 'complaint forwarded'}

In [53]:
train_test['test']['label'][99:100]

[1]

In [59]:
complaint = train_test['test']['text'][100:101]

complaint


['i was shocked when i reviewed my credit report and found late payment on the dates below   days past due as of   i am not sure how this happened i believe that i had made my payments to you when i received my statements my only thought is that my monthly statement did not get to me']

In [58]:
complaint = train_test['test']['text'][100:101]

forward_complaint(complaint)

KeyboardInterrupt: 